In [ ]:
!pip install keras # install the keras module
import keras # import the keras module

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
import numpy as np

In [ ]:
keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


((array([[[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         ...,
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0

In [ ]:
# Check if TPU is available and define tpu within the current scope
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # Get TPU resolver
    print('Running on TPU ', tpu.master())
except ValueError:
    print('TPU not found.')
    tpu = None  # Assign None to tpu if not found


Running on TPU  


In [ ]:
# Connect to TPU if available
if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)  # Define TPU strategy
else:
    print("Running on CPU/GPU since TPU was not found.")

In [ ]:
# Load the dataset
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [ ]:
# Preprocess the data
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255

In [ ]:
# Built the model
model=models.Sequential()

In [ ]:
# Convolutional Layer 1
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(28,28,1),padding='same'))


In [ ]:
# Max Pooling Layer 1
model.add(layers.MaxPooling2D((2, 2)))

In [ ]:
# Convolutional Layer 2
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [ ]:
# Max Pooling Layer 2
model.add(layers.MaxPooling2D((2, 2)))

In [ ]:
# Flatten the output
model.add(layers.Flatten())

In [ ]:
model.add(Dense(128,activation='relu'))

In [ ]:
# Output Layer
model.add(layers.Dense(10, activation='softmax'))

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(train_images, train_labels, epochs=10, batch_size=64, validation_split=0.1)


Epoch 1/10
844/844 [==============================] - 9s 10ms/step - loss: 0.4564 - accuracy: 0.8347 - val_loss: 0.3352 - val_accuracy: 0.8770
Epoch 2/10
844/844 [==============================] - 8s 9ms/step - loss: 0.3003 - accuracy: 0.8897 - val_loss: 0.2861 - val_accuracy: 0.8973
Epoch 3/10
844/844 [==============================] - 8s 9ms/step - loss: 0.2551 - accuracy: 0.9062 - val_loss: 0.2698 - val_accuracy: 0.9022
Epoch 4/10
844/844 [==============================] - 8s 9ms/step - loss: 0.2240 - accuracy: 0.9175 - val_loss: 0.2427 - val_accuracy: 0.9117
Epoch 5/10
844/844 [==============================] - 8s 9ms/step - loss: 0.1985 - accuracy: 0.9273 - val_loss: 0.2293 - val_accuracy: 0.9153
Epoch 6/10
844/844 [==============================] - 8s 9ms/step - loss: 0.1759 - accuracy: 0.9348 - val_loss: 0.2164 - val_accuracy: 0.9262
Epoch 7/10
844/844 [==============================] - 8s 9ms/step - loss: 0.1580 - accuracy: 0.9408 - val_loss: 0.2092 - val_accuracy: 0.9257
Epoch

In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_acc}")

313/313 [==============================] - 1s 3ms/step - loss: 0.2456 - accuracy: 0.9226
Test accuracy: 0.9225999712944031


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 12, 12, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 6, 6, 64)          0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 2304)              0         
                                                                 
 dense (Dense)               (None, 128)               2

In [ ]:
model.save('fashion_mnist_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
